<a href="https://colab.research.google.com/github/hws2002/Deep_Learning_with_Keras/blob/main/Chapter7/Chapter7_4_fit_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 훈련 vs 추론

In [1]:
def train_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training = True)
    loss = loss_fn(targets, predictions)
  gradients = tape.gradients(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(model.trainable_Weights, gradients))


# 측정 지표의 저수준 사용법

In [2]:
import keras
from keras import layers

metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0,1,2]
predictions = [[1,0,0],[0,1,0],[0,0,1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"결과 : {current_result:.2f}")

결과 : 1.00


In [3]:
values = [0,1,2,3,4]

mean_tracker = keras.metrics.Mean()
for value in values:
  mean_tracker.update_state(value)
print(f"평균 지표 : {mean_tracker.result():.2f}")

평균 지표 : 2.00


# 완전한 훈련과 평가 루프

In [4]:

def get_mnist_model():
  inputs = keras.Input(shape = (28*28,))
  features = layers.Dense(512, activation = "relu")(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation = "softmax")(features)
  model =  keras.Model(inputs, outputs)
  return model

model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

def train_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training = True)
    loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss ,model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:
      metric.update_state(targets, predictions)
      logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss) # 손실 평균을 계산함
    logs['loss'] = loss_tracking_metric.result()
    return logs

매 에포크 시작과 평가 전에 지표의 상태를 재설정해야 함.

In [5]:
def reset_metrics():
  for metric in metrics:
    metric.reset_state()
  loss_tracking_metric.reset_state()

이제 완전한 훈련 루프를 구성해보자

In [6]:
from tensorflow.keras.datasets import mnist

(images,labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28*28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
import tensorflow as tf
training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)
)

training_dataset = training_dataset.batch(32)
epochs = 3
for epoch in range(epochs):
  reset_metrics()
  for inputs_batch, targets_batch in training_dataset:
    logs = train_step(inputs_batch, targets_batch)
  print(f"{epoch}번째 에포크 결과")
  for key, value in logs.items():
    print(f"...{key} : {value:.4f}")

0번째 에포크 결과
...sparse_categorical_accuracy : 0.9122
...loss : 0.2939
1번째 에포크 결과
...sparse_categorical_accuracy : 0.9545
...loss : 0.1598
2번째 에포크 결과
...sparse_categorical_accuracy : 0.9636
...loss : 0.1289


## 평가 루프

In [8]:

def test_step(inputs, targets):
  predictions = model(inputs, training = False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs['val_' + metric.name] = metric.result()
  loss_tracking_metric.update_state(loss)
  logs['val_loss'] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()

for inputs_batch, targets_batch in val_dataset:
  logs = test_step(inputs_batch, targets_batch)

print("평가 결과 : ")
for key, value in logs.items():
  print(f"...{key} : {value:.4f}")

평가 결과 : 
...val_sparse_categorical_accuracy : 0.9675
...val_loss : 0.1228


# tf.function으로 성능 높이기

In [9]:
# @tf.function # 이 라인만 추가됨
def test_step(inputs, targets):
  predictions = model(inputs, training = False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs['val_' + metric.name] = metric.result()

  loss_tracking_metric.update_state(loss)
  logs['val_loss'] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()

for inputs_batch, targets_batch in val_dataset:
  logs = test_step(inputs_batch, targets_batch)

print("평가 결과 : ")
for key, value in logs.items():
  print(f"...{key} : {value:.4f}")

평가 결과 : 
...val_sparse_categorical_accuracy : 0.9675
...val_loss : 0.1228


# fit() 메서드를 사용자 정의 루프로 활용하기

In [19]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()

loss_tracker = keras.metrics.Mean(name = "loss") # 이 객체는 훈련과 평가 과정에서 배치 손실의 평균을 추적함

class CustomModel(keras.Model):
  def train_step(self, data): # train_step 메서드를 오버라이딩함
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training = True) # 모델이 클래스 자체이므로 model(inputs, training = True) 대신에
      # self(inputs, training = True)를 사용함
      loss = loss_fn(targets, predictions)

    gradients = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
    loss_tracker.update_state(loss) # 손실의 평균을 추적하는 loss_tracker를 업데이트함
    return {"loss" : loss_tracker.result()} # 평균 손실을 구함

  @property
  def metrics(self): # 에포크마다 재설정할 지표는 여기에 나열해야 함
    return [loss_tracker]

이제 사용자 정의 모델의 객체를 만들고 컴파일하고 (손실은 모델 밖에서 이미 정의했기 때문에 옵티마이저만 전달함) 보통 떄처럼 fit() 메서드로 훈련할 수 있음

In [20]:
inputs = keras.Input(shape = (28*28,))
features = layers.Dense(512, activation = 'relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation = "softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(
    optimizer = keras.optimizers.RMSprop()
)

model.fit(train_images, train_labels, epochs = 3)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.4452
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 0.1656
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - loss: 0.1296


이제 compile() 메서드를 통해 지표와 손실을 설정하면 어떨까?

compile() 메서드를 호출한 후 다음을 참조할 수 있음

- self.compiled_loss : compile() 메서드에 전달할 손실 함수
- self.compiled_metrics : compile() 메서드에 전달된 지표 목록이 포함되어 있는 객체. selfe.compiled_metrics.update_state()를 호출하여 모든 지표를 동시에 업데이트 할 수 있음.
- self.metrics : compile() 메서드에 전달한 실제 지표의 목록. 앞서 loss_tracking_metric으로 수동으로 했던 것과 비슷하게 손실을 추적하는 지표도 포함함

따라서 다음과 같이 쓸 수 있음

In [ ]:

class CustomModel(keras.Model):
  def train_Step(self, data): # train_step 메서드를 오버라이딩함
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training = True) # 모델이 클래스 자체이므로 model(inputs, training = True) 대신에
      # self(inputs, training = True)를 사용함
      loss = self.compiled_loss(targets, predictions) # self.compiled_metrics 모델의 지표를 업데이트함

    gradients = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradient(zip(gradients, self.trainable_weights))
    self.compiled_metrics.update_state(loss) # self.compiled_loss를 사용해서 손실을 계산
    return {m.name() : m.result() for m in self.metrics} # 측정 지표 이름과 현재 값을 매핑한 딕셔너리를 반환

  @property
  def metrics(self): # 에포크마다 재설정할 지표는 여기에 나열해야 함
    return [loss_tracker]

테스트해보자

In [23]:
inputs = keras.Input((28*28,))
features = layers.Dense(512, activation = "relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation = "softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = [keras.metrics.SparseCategoricalAccuracy()])

model.fit(train_images, train_labels, epochs = 3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 0.4543
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.1648
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.1260
